In [1]:
"""절대 모멘텀 전략"""

import pandas as pd
import numpy as np
import datetime
import FinanceDataReader as fdr

df = fdr.DataReader('AAPL', '1996')
price_df = df.loc[:,['Close']].copy()
price_df.rename(columns={'Close' : 'Adj Close'}, inplace=True)
price_df['Date'] = price_df.index

price_df.head()

,Adj Close,Date
Date,,
1996-01-02,0.29,1996-01-02
1996-01-03,0.29,1996-01-03
1996-01-04,0.28,1996-01-04
1996-01-05,0.31,1996-01-05
1996-01-08,0.31,1996-01-08


In [2]:
# 월말 데이터 추가
price_df['STD_YM'] = price_df['Date'].map(lambda x : datetime.datetime.strftime(x, '%Y-%m'))
price_df = price_df[price_df['Date'] > '2010-03-30']
price_df.head()

,Adj Close,Date,STD_YM
Date,,,
2010-03-31,8.39,2010-03-31,2010-03
2010-04-01,8.43,2010-04-01,2010-04
2010-04-05,8.52,2010-04-05,2010-04
2010-04-06,8.55,2010-04-06,2010-04
2010-04-07,8.59,2010-04-07,2010-04


In [3]:
# 월말 종가 데이터프레임
month_list = price_df['STD_YM'].unique()
month_last_df = pd.DataFrame()
for m in month_list:
    month_last_df = month_last_df.append(price_df.loc[price_df[price_df['STD_YM'] == m].index[-1], : ])

month_last_df.set_index(['Date'], inplace=True)
month_last_df.head()

,Adj Close,STD_YM
Date,,
2010-03-31,8.39,2010-03
2010-04-30,9.32,2010-04
2010-05-28,9.17,2010-05
2010-06-30,8.98,2010-06
2010-07-30,9.19,2010-07


In [4]:
# 1개월 전 말일자 종가, 12개월 전 말일자 종가 가져오기
month_last_df['BF_1M_Adj Close'] = month_last_df['Adj Close'].shift(1)
month_last_df['BF_12M_Adj Close'] = month_last_df['Adj Close'].shift(12)
month_last_df.fillna(0, inplace=True)
month_last_df.head(15)

,Adj Close,STD_YM,BF_1M_Adj Close,BF_12M_Adj Close
Date,,,,
2010-03-31,8.39,2010-03,0.00,0.00
2010-04-30,9.32,2010-04,8.39,0.00
2010-05-28,9.17,2010-05,9.32,0.00
2010-06-30,8.98,2010-06,9.17,0.00
2010-07-30,9.19,2010-07,8.98,0.00
2010-08-31,8.68,2010-08,9.19,0.00
2010-09-30,10.13,2010-09,8.68,0.00
2010-10-29,10.75,2010-10,10.13,0.00
2010-11-30,11.11,2010-11,10.75,0.00


In [5]:
# 포지션 기록
book = price_df.copy()
book.set_index(['Date'], inplace=True)
book['trade'] = ''
book.head()

,Adj Close,STD_YM,trade
Date,,,
2010-03-31,8.39,2010-03,
2010-04-01,8.43,2010-04,
2010-04-05,8.52,2010-04,
2010-04-06,8.55,2010-04,
2010-04-07,8.59,2010-04,


In [6]:
# 모멘텀이 발생하면 매수 신호가 발생하도록 함
ticker = 'SPY'
for x in month_last_df.index:
    signal = ''
    flag = False
    momentum = month_last_df.loc[x, 'BF_1M_Adj Close'] / month_last_df.loc[x, 'BF_12M_Adj Close'] - 1
    if momentum > 0.0 and momentum != np.inf and momentum != -np.inf:
        signal = 'buy ' + ticker
        flag = True
    book.loc[x:, 'trade'] = signal
    print('날짜: ', x, ' 모멘텀 인덱스: ', momentum, ' flag: ', flag, ' signal: ', signal)

<ipython-input-6-8ad1ce970292>:6: RuntimeWarning: invalid value encountered in double_scalars
  momentum = month_last_df.loc[x, 'BF_1M_Adj Close'] / month_last_df.loc[x, 'BF_12M_Adj Close'] - 1
<ipython-input-6-8ad1ce970292>:6: RuntimeWarning: divide by zero encountered in double_scalars
  momentum = month_last_df.loc[x, 'BF_1M_Adj Close'] / month_last_df.loc[x, 'BF_12M_Adj Close'] - 1


날짜:  2010-03-31 00:00:00  모멘텀 인덱스:  nan  flag:  False  signal:  
날짜:  2010-04-30 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2010-05-28 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2010-06-30 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2010-07-30 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2010-08-31 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2010-09-30 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2010-10-29 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2010-11-30 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2010-12-31 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2011-01-31 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2011-02-28 00:00:00  모멘텀 인덱스:  inf  flag:  False  signal:  
날짜:  2011-03-31 00:00:00  모멘텀 인덱스:  0.5029797377830749  flag:  True  signal:  buy SPY
날짜:  2011-04-29 00:00:00  모멘텀 인덱스:  0.3358369098712446  flag:  True  signal:  buy SPY
날짜:  2011-05-31 00:00:00  모멘텀 인덱스:  0.3631406761

In [7]:
# 전략 수익률 -> 평균 회귀 전략할 때와 거의 비슷함!
def returns(book , ticker):
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    
    for i in book.index:
        if book.loc[i, 'trade'] == 'buy ' + ticker and book.shift(1).loc[i,'trade'] == '':
            buy = book.loc[i, 'Adj Close']
            print('진입일: ', i, ' 진입가격: ', buy)
        elif book.loc[i, 'trade'] == 'buy ' + ticker and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            current = book.loc[i, 'Adj Close']
            rtn = (current - buy) / buy + 1
            book.loc[i, 'return'] = rtn
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            sell = book.loc[i, 'Adj Close']
            rtn = (sell - buy) / buy + 1
            book.loc[i, 'return'] = rtn
            print('청산일: ', i, ' 진입가격: ', buy, ' | long 청산가격: ', sell, 'return: ', round(rtn, 4))
        
        if book.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0
            current = 0.0
    
    acc_rtn = 1.0
    for i in book.index:
        if book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy ' + ticker:
            rtn = book.loc[i, 'return']
            acc_rtn = acc_rtn * rtn
            book.loc[i, 'acc return'] = acc_rtn
    
    print('Accumulated return: ', round(acc_rtn, 4))
    return round(acc_rtn, 4)

print(returns(book, ticker))

진입일:  2011-03-31 00:00:00  진입가격:  12.45
청산일:  2013-02-28 00:00:00  진입가격:  12.45  | long 청산가격:  15.76 return:  1.2659
진입일:  2013-12-31 00:00:00  진입가격:  20.04
청산일:  2016-01-29 00:00:00  진입가격:  20.04  | long 청산가격:  24.34 return:  1.2146
진입일:  2016-12-30 00:00:00  진입가격:  28.95
청산일:  2019-01-31 00:00:00  진입가격:  28.95  | long 청산가격:  41.61 return:  1.4373
진입일:  2019-03-29 00:00:00  진입가격:  47.49
청산일:  2019-06-28 00:00:00  진입가격:  47.49  | long 청산가격:  49.48 return:  1.0419
진입일:  2019-07-31 00:00:00  진입가격:  53.26
청산일:  2019-08-30 00:00:00  진입가격:  53.26  | long 청산가격:  52.19 return:  0.9799
진입일:  2019-10-31 00:00:00  진입가격:  62.19
Accumulated return:  2.2562
2.2562
